<a href="https://colab.research.google.com/github/realmistic/PythonInvest-basic-fin-analysis/blob/master/_Part4_%5B2024%5D_Scrapping_EPS_Q2_20_and_Daily_Returns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The grand idea**: check the financials reporting vs. stock performance.

**Additional Links**:
* Company guidance on earnings: https://www.investopedia.com/terms/g/guidance.asp
* Price to earnings ratio: https://www.investopedia.com/terms/p/price-earningsratio.asp

* Trading Revolut: https://www.revolut.com/legal/trading
* (For next articles) New features Revolut: Out of Hours Orders, Limit Orders, Stop Loss Orders


**ARTICLE OUTLINE**

**Intro** The stocks ownership gives you the right to share in the profits of the company ([wiki article](https://en.wikipedia.org/wiki/Common_stock)). In an ideal world it should be highly dependant on the earnings of the company, as the price of the stock is a discounted future profits. If at some point a company earns more than previously - it might mean that the growth of a company is accelerated and the stock should be priced higher.
That's why among other financials to follow - earnings per share (EPS) is one of the most important one. Every quarter analysts make predictions on the company profits (or losses) and then check those predictions vs. actual reported EPS.
If the company is doing better than predicted - it should cause the stocks price increast, and vice versa.

In this article we aim to test this at scale - for hundreds of stocks that reported earnings in 2020 Q2. We will check the dependency of a stock's price fluctuation vs. actual EPS, predicted EPS, and Surprise

**Part 0** Describe scraping - HTTP request with Params, Chrome Dev tools - check DOM (span, table, tr, tags) and apply that with Beautiful Soup

**Part 4.1** Describe 1 day financials for Facebook. Important caveat : reporting date is wrong on the website (it is 1 day earlier - for the most of large stocks)

**Part 1** Pack everything in 1 function to get a weekly stats on the dates and EPS

**Part 4.2** Get stock returns and volume for all found tickers in Yahoo finance -> join the datasets on a Ticker (Symbol) key.

**Part 5** The distribution of returns and the visualisation


**Conclusion** We've shown how to scrape the financial predictions from a website and how to connect them together with the stock returns. Q2'20 seems to be a very successfull quarter for the top 50 (on volume trade) stocks - most of them showing the positive surprise over the expected earnings-per-share (EPS) and high short-term returns. The result remains strong even after the corresponding S&P500 index returns are deducted (i.e. the top 50 stocks had higher positive growth than average index dynamics).
When scaled to top-200 stocks - the result is not that simple - the average returns are smaller, and there is more variation in EPS and the returns.


# 0) Scrape ONE PAGE data on stocks financials (earnings-per-share) : STEP-BY-STEP

In [ ]:
!pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
url = "https://finance.yahoo.com/calendar/earnings?from=2020-07-26&to=2020-08-01&day=2020-07-27"

In [ ]:
r = requests.get(url)

In [ ]:
r.ok

True

In [ ]:
r.status_code

200

In [ ]:
r.headers

{'referrer-policy': 'no-referrer-when-downgrade', 'strict-transport-security': 'max-age=15552000', 'x-frame-options': 'SAMEORIGIN', 'content-security-policy': 'sandbox allow-downloads allow-forms allow-modals allow-same-origin allow-scripts allow-popups allow-popups-to-escape-sandbox allow-top-navigation-by-user-activation allow-presentation;', 'content-type': 'text/html; charset=utf-8', 'vary': 'Accept-Encoding', 'content-encoding': 'gzip', 'set-cookie': 'B=3fedq7dg2gk7u&b=3&s=5q; expires=Sat, 13-Feb-2022 22:21:50 GMT; path=/; domain=.yahoo.com', 'date': 'Sat, 13 Feb 2021 22:21:50 GMT', 'x-envoy-upstream-service-time': '662', 'server': 'ATS', 'x-envoy-decorator-operation': 'finance-nodejs--default-production-bf1.finance-k8s.svc.yahoo.local:4080/*', 'Age': '0', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Expect-CT': 'max-age=31536000, report-uri="http://csp.yahoo.com/beacon/csp?src=yahoocom-expect-ct-report-only"', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-O

In [ ]:
r.content

b'<!DOCTYPE html><html id="atomic" class="NoJs desktop" lang="en-US"><head prefix="og: http://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark(\'PageStart\');</script><meta charset="utf-8"/><title>Company Earnings Calendar - Yahoo Finance</title><meta name="keywords" content="Earnings Announcements, Company Earnings, EPS Estimates, Earnings Per Share, Conference Calls, Before Market Open, After Market Close"/><meta http-equiv="x-dns-prefetch-control" content="on"/><meta property="twitter:dnt" content="on"/><meta property="fb:app_id" content="458584288257241"/><meta name="theme-color" content="#400090"/><meta name="viewport" content="width=device-width, initial-scale=1"/><meta name="description" lang="en-US" content="Learn when companies announce their quarterly and annual earnings, along with the latest EPS estimates and conference call times from Yahoo Finance."/><meta name="oath:guce:consent-host" content="guce.yahoo.com"/><meta name="msval

In [ ]:
soup = BeautifulSoup(r.text)

In [ ]:
table = soup.find_all('table')

In [ ]:
# Just 1 table found which is good
len(table)

1

In [ ]:
# Get all column names
# spans = soup.table.thead.find_all('span')

# Apr-2024: webpage markup has changed since 2021
spans = soup.table.thead.find_all('th')

In [ ]:
columns = []
for span in spans:
  print(span.text)
  columns.append(span.text)

Symbol
Company
Earnings Call Time
EPS Estimate
Reported EPS
Surprise(%)


In [ ]:
rows = soup.table.tbody.find_all('tr')

In [ ]:
# 100 rows in the table
len(rows)

100

In [ ]:
# read row by row
stocks_df = pd.DataFrame(columns=columns)

for row in rows:
  elems = row.find_all('td')
  dict_to_add = {}
  for i,elem in enumerate(elems):
    dict_to_add[columns[i]] = elem.text
    # if i<=2:
    #   dict_to_add[columns[i]]=elem.text
    # else:
    #   dict_to_add[columns[i]]= float(elem.text)
  stocks_df = stocks_df.append(dict_to_add, ignore_index=True)

In [ ]:
stocks_df

,Symbol,Company,Earnings Call Time,EPS Estimate,Reported EPS,Surprise(%)
0,MMM,3M Company,Time Not Supplied,1.8,1.78,-1.17
1,BGFV,Big 5 Sporting Goods Corporation,Time Not Supplied,-,0.52,-
2,ABG,"Asbury Automotive Group, Inc.",Time Not Supplied,2.49,2.52,+1.16
3,WEIIF,Wolverine Energy and Infrastructure Inc.,Time Not Supplied,0.03,-0.05,-273.33
4,AFL,Aflac Incorporated,Time Not Supplied,1.07,1.28,+20.08
...,...,...,...,...,...,...
95,MASI,Masimo Corporation,Time Not Supplied,0.81,0.85,+4.68
96,SID,Companhia Siderúrgica Nacional,Time Not Supplied,-0.31,0.25,+180.91
97,GCWOF,"Grupo Cementos de Chihuahua, S.A.B. de C.V.",Time Not Supplied,0.06,0.11,+100
98,NSANY,"Nissan Motor Co., Ltd.",Time Not Supplied,-,-,-


In [ ]:
# 100 ROWS scraped and 6 columns. We will use all columns, but Earnings Call Time as it is not supplied in many cases
  # There are some problems in the dataset:
  #  - missing values, where EPS Estimate, Reported EPS, or Surprise are unknown
  #  - need to convert numetic values to float to work with them

stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Symbol              100 non-null    object
 1   Company             100 non-null    object
 2   Earnings Call Time  100 non-null    object
 3   EPS Estimate        100 non-null    object
 4   Reported EPS        100 non-null    object
 5   Surprise(%)         100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB


# 1) Pack everything in one scrape function
* it returns stocks_df over 1 week
* call the function during the several weeks period

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as nm
from datetime import datetime, timedelta, date

In [ ]:
# Need to supply weekly stats as you see on the website
# from_dt = '2020-07-26'
# to_dt = '2020-08-01'

def get_scrapped_week(from_dt, to_dt):

  # initially look at the first 100 stocks with earnings at the first day of the week (from_dt)
  # FULL URL with PARAMS example: url = "https://finance.yahoo.com/calendar/earnings?from=2020-07-26&to=2020-08-01&day=2020-07-27"
  url = "https://finance.yahoo.com/calendar/earnings"
  offset = 0
  size = 100
  fst = 1

  # scrape every date in the submitted interval
  for day_date in (datetime.strptime(from_dt, '%Y-%m-%d')  + timedelta(n) for n in range(6)):
    day_dt = datetime.strftime(day_date, '%Y-%m-%d')
    print(day_dt)

    # inner cycle for iteration with offset, if more than 100 stocks earnings happenned that date
    while True:
      # make URL request with the params
      params = {'from': from_dt, 'to': to_dt,'day': day_dt, 'offset':offset, 'size': size}
      r = requests.get(url, params=params)
      soup = BeautifulSoup(r.text)

      # scrape table column names when going first time to create a correct dataframe
      if fst == 1:
        spans = soup.table.thead.find_all('span')
        columns = []
        for span in spans:
          print(span.text)
          columns.append(span.text)
        stocks_df = pd.DataFrame(columns=columns)
        fst = 0

      # scrape body with row values
      rows = soup.table.tbody.find_all('tr')
      for row in rows:
        elems = row.find_all('td')
        dict_to_add = {}
        dict_to_add['Date'] = day_dt
        for i,elem in enumerate(elems):
          dict_to_add[columns[i]]=elem.text
        stocks_df = stocks_df.append(dict_to_add, ignore_index=True)
      if len(rows) != 100:
        print(len(rows)+offset)
        offset = 0
        break
      else:
        offset = offset + 100

  return stocks_df

# stocks_df.to_csv('stocks.csv', index = False)

In [ ]:
stocks_df = get_scrapped_week('2020-07-05', '2020-07-11')

2020-07-05
Symbol
Company
Earnings Call Time
EPS Estimate
Reported EPS
Surprise(%)
13
2020-07-06
23
2020-07-07
27
2020-07-08
37
2020-07-09
26
2020-07-10
1


In [ ]:
stocks_df.head()

,Symbol,Company,Earnings Call Time,EPS Estimate,Reported EPS,Surprise(%),Date
0,AIABF,AirAsia Group Berhad,Time Not Supplied,-0.01,-0.15,-2400,2020-07-05
1,JNEXF,Cuda Oil and Gas Inc.,Time Not Supplied,-0.07,-0.26,-271.43,2020-07-05
2,SINO,"Sino-Global Shipping America, Ltd.",Time Not Supplied,-,-,-,2020-07-05
3,AXAS,Abraxas Petroleum Corporation,Time Not Supplied,-0.14,-0.8,-471.43,2020-07-05
4,BSTJF,Boustead Projects Limited,Time Not Supplied,-,-,-,2020-07-05


In [ ]:
stocks_df = stocks_df.append(get_scrapped_week('2020-07-12', '2020-07-18'))

2020-07-12
Symbol
Company
Earnings Call Time
EPS Estimate
Reported EPS
Surprise(%)
28
2020-07-13
52
2020-07-14
63
2020-07-15
68
2020-07-16
80
2020-07-17


AttributeError: ignored

In [ ]:
stocks_df = stocks_df.append(get_scrapped_week('2020-07-19', '2020-07-25'))

In [ ]:
stocks_df = stocks_df.append(get_scrapped_week('2020-07-26', '2020-08-01'))

In [ ]:
stocks_df.info()

# 3) Clean up : no missing values and transform text to float

In [ ]:
filter1 = stocks_df['Surprise(%)']!='-'
filter2 = stocks_df['EPS Estimate']!='-'
filter3 = stocks_df['Reported EPS']!='-'

stocks_df_noMissing = stocks_df[filter1 & filter2 & filter3]

In [ ]:
# Around 20% had missing values
stocks_df_noMissing.info()

In [ ]:
# Convert to float numeric values
stocks_df_noMissing['EPS Estimate'] = stocks_df_noMissing['EPS Estimate'].astype(float)
stocks_df_noMissing['Reported EPS'] = stocks_df_noMissing['Reported EPS'].astype(float)
stocks_df_noMissing['Surprise(%)'] = stocks_df_noMissing['Surprise(%)'].astype(float)

In [ ]:
# stocks_df_noMissing['Date'] = pd.to_datetime(stocks_df_noMissing['Date'])

# stocks_df_noMissing['Date'] = stocks_df_noMissing['Date'].astype(date)

In [ ]:
stocks_df_noMissing.info()

In [ ]:
stocks_df_noMissing.set_index('Symbol')

In [ ]:
stocks_df_noMissing[stocks_df_noMissing['Symbol']=='GOOGL']

# 4) Add stocks price for the companies

In [ ]:
!pip install yfinance

## 4.1) Try one stock one date --> calc returns and volume jump

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [ ]:
# Test dates
date = datetime.strptime('2020-08-08', '%Y-%m-%d')
print(date + timedelta(days=1))
print(date - timedelta(days=1))

In [ ]:
# IMPORTANT!!! For some reason the website has incorrect data for the day (one day earniere)
# https://abc.xyz/investor/static/pdf/2020Q2_alphabet_earnings_release.pdf?cache=a881c38
stocks_df_noMissing[stocks_df_noMissing['Symbol']=='GOOGL']

In [ ]:
# https://s21.q4cdn.com/399680738/files/doc_financials/2020/q2/Q2'20-FB-Financial-Results-Press-Release.pdf
# SAME! date is wrong in the datasource
stocks_df_noMissing[stocks_df_noMissing['Symbol']=='FB']

In [ ]:
# https://seekingalpha.com/symbol/AMZN/earnings. -- announce date one day later
stocks_df_noMissing[stocks_df_noMissing['Symbol']=='AMZN']

In [ ]:
stocks_df_noMissing.info()

In [ ]:
row = stocks_df_noMissing[stocks_df_noMissing['Symbol']=='FB']
date = row['Date'].values[0]

In [ ]:
print(date)

In [ ]:
date = datetime.strptime(row['Date'].values[0], '%Y-%m-%d')

print(date + timedelta(days=3))
print(date - timedelta(days=1))

ticker = yf.Ticker('FB')

# https://pypi.org/project/yfinance/
hist = yf.download("FB", start= date - timedelta(days=1), end=date + timedelta(days=3))

In [ ]:
hist

In [ ]:
# 2 days returns -> should be defined only for the last observation
hist['r2'] = np.log(hist['Open'] / hist['Open'].shift(2))
hist['volume_rise'] = np.log(hist['Volume'] / hist['Volume'].shift(2))


In [ ]:
# exp(0.10)=1.10 or 10% INCREASE, AND exp(1.36)=3.9x VOLUME RISE OF TRADE FOR FACEBOOK
hist

In [ ]:
# The last value of returns (r2 = 2-days return on the next morning after the event) in the dataframe
hist.r2.values[-1]

In [ ]:
# Volume of trade rising
hist.volume_rise.values[-1]

## 4.15) Get S&P 500 stats

In [ ]:
import pandas_datareader.data as pdr
# from datetime import datetime
from datetime import date
# end = date.today()
# start = datetime(year=end.year, month=end.month-1, day=end.day)

start = datetime(2020,7,1)
end = datetime(2020,8,10)
print(f'Period 1 year until today: {start} to {end} ')

spx_index = pdr.get_data_stooq('^SPX', start, end)

In [ ]:
# S&P500 index was growing almost all July 2020 --> need to adjust stock growth after the reporting date
spx_index['Open'].plot.line()

In [ ]:
# 2 days returns daily in July-Aug 2020

spx_index['r2'] = np.log(np.divide(spx_index['Open'] , spx_index['Open'].shift(2)))
spx_index['r2'].plot.line()

In [ ]:
spx_index.head(30)

In [ ]:
# Filling an array of S&P return for corresponding stock
# Important: if there is a "gap" for that particular date -> we take the closest previous value
array_returns_snp500 = []
for index,row in stocks_df_noMissing.iterrows():
  start_dt = datetime.strptime(row['Date'], '%Y-%m-%d') - timedelta(days = 1)
  end_dt = datetime.strptime(row['Date'], '%Y-%m-%d') + timedelta(days = 3)
  # we don't have gaps more than 4 days -> try to find the closest value of S&P500 returns in the dataframe:
  cur_dt = end_dt
  while cur_dt >= start_dt:
    rez_df = spx_index[cur_dt.strftime('%Y-%m-%d')]
    if len(rez_df)>0:
      array_returns_snp500.append(rez_df.r2.values[0])
      break
    else:
      cur_dt = cur_dt - timedelta(days = 1)

In [ ]:
len(array_returns_snp500)

In [ ]:
len(stocks_df_noMissing)

## 4.2) Get all stocks returns and volume rise

In [ ]:
# 2 days returns on open price after earnings vs. 2 days ago
array_tickers = []
array_returns = []
array_volume_rise = []
array_volume_usd = []
array_snp500 = []

for index,row in stocks_df_noMissing.iterrows():
  start_dt = datetime.strptime(row['Date'], '%Y-%m-%d') - timedelta(days = 1)
  end_dt = datetime.strptime(row['Date'], '%Y-%m-%d') + timedelta(days = 3)
  hist = yf.download(row['Symbol'], start = start_dt, end = end_dt)
  # We need to have a full data : volume and price for all dates calculate the returns and volume rise
  # ALSO: if end_dt is non-trading day (Sat,Sun) --> we can't directly calc the stats of returns
  if len(hist)<4:
    continue
  hist['r2'] = np.log(np.divide(hist['Open'] , hist['Open'].shift(2)))
  hist['volume_rise'] = np.log(np.divide(hist['Volume'], hist['Volume'].shift(2)))
  hist['volume_usd'] = hist['Volume'] * hist['Open']
  print(row)
  print(index)
  # print(hist)
  print('--------------')
  array_tickers.append(row['Symbol'])
  array_returns.append(hist.r2.values[-1])
  array_volume_rise.append(hist.volume_rise.values[-1])
  array_volume_usd.append(hist.volume_usd.values[-1])
  # We only append values S&P for the stocks that have all the data
  array_snp500.append(array_returns_snp500[index])

In [ ]:
# 964 stocks have financials: volume of trade, volume_rise (in stocks amount), and returns
len(array_tickers)

## 4.4) Merge all together

In [ ]:
returns_df = pd.DataFrame(columns=['Ticker', 'Returns','Volume Rise','Volume Trade USD','Returns S&P500'])

returns_df = pd.DataFrame([array_tickers,array_returns,array_volume_rise,array_volume_usd, array_snp500]).transpose()

returns_df.columns=['Ticker','Returns','Volume Rise','Volume Trade USD', 'Returns S&P500']

returns_df.set_index('Ticker',inplace=True)

returns_df.dropna(inplace=True)

In [ ]:
returns_df['Returns'] = returns_df['Returns'].astype(float)
returns_df['Volume Rise'] = returns_df['Volume Rise'].astype(float)
returns_df['Volume Trade USD'] = returns_df['Volume Trade USD'].astype(float)
returns_df['Returns S&P500'] = returns_df['Returns S&P500'].astype(float)


In [ ]:
returns_df['Returns in %'] = np.exp(returns_df['Returns'])
returns_df['Volume Rise in %'] = np.exp(returns_df['Volume Rise'])

# Returns above S&P500
returns_df['Adj. Returns'] = returns_df['Returns'] - returns_df['Returns S&P500']
returns_df['Adj. Returns in %'] = np.exp(returns_df['Adj. Returns'])


In [ ]:
# if we want to have a hist -> no INF values
returns_df = returns_df.replace([np.inf, -np.inf], np.nan)

In [ ]:
returns_df.hist(figsize=(20,10), bins=100)

In [ ]:
stocks_and_returns = stocks_df_noMissing.set_index('Symbol').join(returns_df)

stocks_and_returns.head()

In [ ]:
stocks_and_returns_no_missing = stocks_and_returns.replace([np.inf, -np.inf], np.nan).dropna()

In [ ]:
stocks_and_returns_no_missing.info()

In [ ]:
# TOP 50 most traded stocks
top50_volume = stocks_and_returns_no_missing.sort_values(by='Volume Trade USD', ascending=False).head(50)

In [ ]:
top50_volume

In [ ]:
top200_volume = stocks_and_returns_no_missing.sort_values(by='Volume Trade USD', ascending=False).head(200)

# 5) Analysis and visualisation

In [ ]:
#TOP 50: Most of the largest stocks try to report with Surprise<100%
top50_volume[['Surprise(%)','Returns in %']].plot.scatter(x='Surprise(%)', y='Returns in %')

In [ ]:
# TOP 200: Similar to top 50 : but "wider around 0 surprise and returns can be higher (up to 30% in 2 days)"
top200_volume[['Surprise(%)','Returns in %']].plot.scatter(x='Surprise(%)', y='Returns in %')

In [ ]:
# TOP 50: Surprise and Reported EPS vs. Returns in %

# HACK : we need subplots to show the axis value
# https://stackoverflow.com/questions/43121584/matplotlib-scatterplot-x-axis-labels
import matplotlib.pyplot as plt
fig, ax = plt.subplots()


top50_volume[['Surprise(%)','Reported EPS','Returns in %']].plot.scatter(x='Reported EPS', y='Surprise(%)', c='Returns in %', colormap='RdYlGn', ax=ax)

In [ ]:
# TOP 200: Surprise and Reported EPS vs. Returns in %
  # Comment: more examples of moderate returns, negative Reported EPS tends to have negative Returns in %

# HACK : we need subplots to show the axis value
# https://stackoverflow.com/questions/43121584/matplotlib-scatterplot-x-axis-labels
fig, ax = plt.subplots()

top200_volume[['Surprise(%)','Reported EPS','Returns in %']].plot.scatter(x='Reported EPS', y='Surprise(%)', c='Returns in %', colormap='RdYlGn', ax=ax)

In [ ]:
# TOP50: Surprise and Reported EPS vs. Adj. Returns in %
  # COMMENT: many stocks have Reported EPS between 0 and 2.5 USD, and positive returns
fig, ax = plt.subplots()


top50_volume[['Surprise(%)','Reported EPS','Adj. Returns in %']].plot.scatter(x='Reported EPS', y='Surprise(%)', c='Adj. Returns in %', colormap='RdYlGn', ax=ax)

In [ ]:
# TOP200: Surprise and Reported EPS vs. Adj. Returns in %
  # COMMENT: less clear picture with many small positive/negative returns
fig, ax = plt.subplots()


top200_volume[['Surprise(%)','Reported EPS','Adj. Returns in %']].plot.scatter(x='Reported EPS', y='Surprise(%)', c='Adj. Returns in %', colormap='RdYlGn', ax=ax)

In [ ]:
# Reported EPS and EPS Estimate vs Returns in %
fig, ax = plt.subplots()

top50_volume.plot.scatter(x='Reported EPS', y='EPS Estimate', c='Returns in %', colormap='RdYlGn', ax=ax)

In [ ]:
# Reported EPS and EPS Estimate vs. Adj.Returns in %
  # COMMENT : similar picture of Adj. Returns vs. Returns

fig, ax = plt.subplots()

top50_volume.plot.scatter(x='Reported EPS', y='EPS Estimate', c='Adj. Returns in %', colormap='RdYlGn', ax=ax)

In [ ]:
# Compare on returns top 50 and top 200 volume-traded stocks
# TOP 200 stocks have more "bell shaped" distribution around 0 and slightly positive returns vs. top 50 stocks
top200_volume['Adj. Returns in %'].hist(bins=50, alpha=0.5)
top50_volume['Adj. Returns in %'].hist(bins=50, alpha = 0.5)

In [ ]:
# AUXILARY GRAPH - maybe don't need to show in the article
# COMMENT: Very high reported EPS lead to high volume trade ()
fig, ax = plt.subplots()


top200_volume[['Adj. Returns in %','Volume Trade USD','Surprise(%)','Reported EPS']].plot.scatter(x='Adj. Returns in %', y='Volume Trade USD', c='Reported EPS', colormap='RdYlGn', ax=ax)
